In [1]:
from datetime import datetime
import fsspec
import matplotlib
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import re
import xarray as xr

from ormhw.core import DATA_DIR

In [2]:
grid_dir = os.path.join(DATA_DIR,'nasa/L2_GRIDDED')

In [3]:
years = range(2015,2024)
local = fsspec.filesystem('file')
for year in years:
    savefp = f"/home/jovyan/data/nasa/oc_{year}.nc"
    if os.path.isfile(savefp):
        continue
    gridded_files = local.glob(grid_dir + f'/oc_{year}*.nc')
    ds_list = []
    for filepath in gridded_files:
        day = re.findall(r'(\d{4}-\d{2}-\d{2})',filepath)[0]
        dt = datetime.strptime(day,'%Y-%m-%d')
        ds = xr.open_dataset(filepath)
        ds = ds.assign_coords({'time':[dt]})
        ds_list.append(ds)
    mds = xr.combine_by_coords(ds_list)
    mds = mds.sel(time = slice(datetime(year,7,9),datetime(year,10,8)))
    mds.to_netcdf(savefp)

In [4]:
ds_list = []
for year in years:
    _ds = xr.open_dataset(f"/home/jovyan/data/nasa/oc_{year}.nc")
    ds_list.append(_ds)
    
mds = xr.combine_by_coords(ds_list)


In [6]:
mds = mds.sel(latitude = slice(39,51),longitude = slice(-128,-123.5))

In [8]:
mds.to_netcdf(f"/home/jovyan/data/nasa/oc_2015-2023.nc")

In [9]:
for year in years:
    _ds = mds.sel(time = slice(datetime(year,1,1), datetime(year,12,31)))
    print(year, len(_ds.time.values))

2015 84
2016 92
2017 92
2018 92
2019 92
2020 89
2021 92
2022 92
2023 85


In [10]:
mds

<xarray.Dataset>
Dimensions:    (latitude: 1200, longitude: 450, time: 810)
Coordinates:
  * latitude   (latitude) float64 39.01 39.02 39.03 39.04 ... 50.98 50.99 51.0
  * longitude  (longitude) float64 -128.0 -128.0 -128.0 ... -123.5 -123.5 -123.5
  * time       (time) datetime64[ns] 2015-07-09 2015-07-10 ... 2023-10-01
Data variables:
    chl        (time, latitude, longitude) float64 nan nan nan ... nan nan nan